In [1]:
from langgraph.graph import END, START, StateGraph
from langchain_ollama import ChatOllama
from typing import TypedDict

In [25]:
# -------------------------
# State
# -------------------------
class BatsmenState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_percent: float
    summary: str

In [26]:
# -------------------------
# Nodes
# -------------------------
def calculate_sr(state: BatsmenState) -> BatsmenState:
    sr = (state["runs"] / state["balls"]) * 100
    return {**state, "sr": sr}


def calculate_bpb(state: BatsmenState) -> BatsmenState:
    boundaries = state["fours"] + state["sixes"]
    bpb = state["balls"] / boundaries if boundaries > 0 else 0
    return {**state, "bpb": bpb}


def calculate_boundary_percent(state: BatsmenState) -> BatsmenState:
    boundary_runs = (state["fours"] * 4) + (state["sixes"] * 6)
    boundary_percent = (boundary_runs / state["runs"]) * 100
    return {**state, "boundary_percent": boundary_percent}


def summary(state: BatsmenState) -> BatsmenState:
    text = f"""
Strike Rate: {state["sr"]:.2f}
Boundary Percentage: {state["boundary_percent"]:.2f}%
Balls Per Boundary: {state["bpb"]:.2f}
"""
    return {**state, "summary": text}

In [27]:
# -------------------------
# Graph building
# -------------------------
graph = StateGraph(BatsmenState)

graph.add_node("calculate_sr", calculate_sr)
graph.add_node("calculate_bpb", calculate_bpb)
graph.add_node("calculate_boundary_percent", calculate_boundary_percent)
graph.add_node("summary", summary)

# Sequential pipeline
graph.add_edge(START, "calculate_sr")
graph.add_edge("calculate_sr", "calculate_bpb")
graph.add_edge("calculate_bpb", "calculate_boundary_percent")
graph.add_edge("calculate_boundary_percent", "summary")
graph.add_edge("summary", END)

workflow = graph.compile()

In [28]:
# -------------------------
# Run
# -------------------------
initial_state = {
    "runs": 100,
    "balls": 50,
    "fours": 7,
    "sixes": 2
}

final_state = workflow.invoke(initial_state)

print(final_state)

{'runs': 100, 'balls': 50, 'fours': 7, 'sixes': 2, 'sr': 200.0, 'bpb': 5.555555555555555, 'boundary_percent': 40.0, 'summary': '\nStrike Rate: 200.00\nBoundary Percentage: 40.00%\nBalls Per Boundary: 5.56\n'}
